In [12]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook,tqdm_pandas
import itertools
import re

In [2]:
path = 'D:/Python/블로그크롤링/output/token/'
file_list = os.listdir(path)

In [3]:
all_df = pd.DataFrame()
for file in tqdm_notebook(file_list):
    read_file = pd.read_csv(path + file)
    all_df = pd.concat([all_df,read_file])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


### 강릉꺼 1개 샘플진행

In [4]:
all_df = all_df.reset_index(drop=True)
all_df = all_df.iloc[:,2:]

In [5]:
all_df = all_df.iloc[:81699]

In [6]:
origin_df = pd.read_csv('D:/Python/블로그크롤링/output/크롤링/통합/강릉_통합_81699.csv')

In [7]:
all_df = pd.concat([origin_df,all_df],axis=1)

In [8]:
# all_df.to_csv('강릉_토큰_81699.csv',index= False)

In [86]:
all_df = pd.read_csv('D:/Python/블로그크롤링/output/token_통합/강릉_토큰_81699.csv')

### 연도별 gap 확인

In [87]:
all_df = all_df.fillna("")
all_df['Noun'] = all_df['Noun'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))
all_df['Adjective'] = all_df['Adjective'].apply(lambda x : re.sub("[\[\]' ]","",x).split(','))

In [165]:
#중복단어를 제거하지 않고 카운트
noun_df = pd.DataFrame()
adj_df = pd.DataFrame()

for s_year in range(2010,2020):
    word_year_df = all_df[all_df['year'] == s_year]

    #명사
    target_df = list(itertools.chain(*word_year_df['Noun']))
    counted = pd.DataFrame(pd.Series(target_df).value_counts()).reset_index()
    counted.columns = ['name','value']
    
    #제거단어 현재단어 강릉
    drop_row = counted[counted['name'].isin(['강릉','강릉시','강원'])].index
    counted.drop(drop_row,inplace = True)
    counted = counted.head(15)
    
    counted['year'] = s_year
    counted['rank'] = counted.value.rank(ascending = False)
    noun_df = pd.concat([noun_df,counted])
    
    #형용사
    target_df = list(itertools.chain(*word_year_df['Adjective']))
    counted = pd.DataFrame(pd.Series(target_df).value_counts()).reset_index()
    counted.columns = ['name','value']
    
    #제거단어 
    drop_row = counted[counted['name'].isin(['이다'])].index
    counted.drop(drop_row,inplace = True)
    counted = counted.head(15)
    
    counted['year'] = s_year
    counted['rank'] = counted.value.rank(ascending = False)
    adj_df = pd.concat([adj_df,counted])

### gap

In [ ]:
#명사
noun_df['gap'] = 0
for one_year in range(2011,2020):
    names = noun_df.loc[(noun_df['year'] == one_year),"name"].values
    for name in names:
        try:
            new_rank = noun_df.loc[(noun_df['year'] == one_year)&(noun_df['name']==name),'rank'].values[0]
            old_rank = noun_df.loc[(noun_df['year'] == one_year-1)&(noun_df['name']==name),'rank'].values[0]
            noun_df.loc[(noun_df['year'] == one_year)&(noun_df['name']==name),'gap'] = old_rank-new_rank
        except :
            noun_df.loc[(noun_df['year'] == one_year)&(noun_df['name']==name),'gap'] = "NEW"
#형용사
adj_df['gap'] = 0
for one_year in range(2011,2020):
    names = adj_df.loc[(adj_df['year'] == one_year),"name"].values
    for name in names:
        try:
            new_rank = adj_df.loc[(adj_df['year'] == one_year)&(adj_df['name']==name),'rank'].values[0]
            old_rank = adj_df.loc[(adj_df['year'] == one_year-1)&(adj_df['name']==name),'rank'].values[0]
            adj_df.loc[(adj_df['year'] == one_year)&(adj_df['name']==name),'gap'] = old_rank-new_rank
        except :
            adj_df.loc[(adj_df['year'] == one_year)&(adj_df['name']==name),'gap'] = "NEW"

noun_df.drop('rank',axis=1,inplace = True)
adj_df.drop('rank',axis=1,inplace = True)

noun_df.to_csv('연도별 명사카운트.csv',encoding = 'cp949',index = 'False')
adj_df.to_csv('연도별 형용사카운트.csv',encoding = 'cp949',index = 'False')

### 중복단어를 제거한다면
* 중요단어들이 오히려 더 잘안나오므로 이보다는 중복단어가 있는편이 나은것으로 추정

In [72]:
# all_df['Noun'] = all_df['Noun'].apply(lambda x : list(set(x)))
# all_df['Adjective'] = all_df['Adjective'].apply(lambda x : list(set(x)))

# for s_year in range(2010,2020):
#     word_year_df = all_df[all_df['year'] == s_year]

#     #명사
#     target_df = list(itertools.chain(*word_year_df['Noun']))
#     counted = pd.DataFrame(pd.Series(target_df).value_counts().head(15)).reset_index()
#     counted.columns = ['name','value']
#     counted['year'] = s_year
#     noun_df = pd.concat([noun_df,counted],axis= 1)
#     #형용사
#     target_df = list(itertools.chain(*word_year_df['Adjective']))
#     counted = pd.DataFrame(pd.Series(target_df).value_counts().head(15)).reset_index()
#     counted.columns = ['name','value']
#     counted['year'] = s_year
#     adj_df = pd.concat([adj_df,counted],axis= 1)

### 확인용

In [42]:
noun_2010[noun_2010['Noun'].apply(lambda x : '배달' in x)].iloc[5]['full_text']

'[" 가끔씩 이런 분들을 만나면 그래도 세상은 살만하구나, 생각하게 된다. 말하자면 양심이 살아있다는 말이 되기도 한다. 하도 험한 뉴스를 많이 보다보니 세상이 다 어떻게 되어 가는것이 아닌가 걱정할 때가 많은데 앞으로는 다 함께 행복해 지도록 좋은 뉴스를 더 많이 찾아내서 살기 좋은 세상임을 알리는 게 좋을 것 같다. \'세상이 이런데 나 하나 쯤\' 하는 막가는 생각을 갖지 않도록.... 내가 나를 용서할 수 있는 삶이 되어야 하지 않을까 양심이 무엇인지를 말해 주는 글이 있어 올려본다. 50년 전 운동화 훔친 60대 노인 속죄하고 싶다 500만원 기부 50년 전 여고에서 운동화 한  켤레를 훔쳤다가 평생 마음속 짐으로 안고 살던 60대 노인이 이 학교를 찾아가 500만원을 기부했다. 지난 1일 오전 10시 강원도 강릉시 강릉여자고등학교 교장실. 남루한 등산복 차림에 모자를 쓴 60대 후반 남성이 교장실로 들어섰다. 노인은 자리에 앉자마자 이 학교 시동진 교장에게 부끄러운 과거를 털어 놓으려고 왔다며 참회하는 표정으로 두툼한 흰색 편지 봉투 하나를 탁자에 꺼내 놓았다. 봉투속에는 은행에서 바로 찾은 듯한 5만원권 지폐 100장이 들어 있었다. 놀란 사 교장이 왜 이러시느냐고 묻자 노인은 1960년대 어려운 집안 형편으로 신문 배달을 하다 강릉여고 복도에 예쁜 운동화가 있어서 한 켤레를 훔쳤다고 말했다. 노인은 그 때 일이 50년 동안 평생 짐이 됐고 죄의식 속에 가슴이 무거워 속죄하는 마음으로 왔다며 욕심 때문에 운동화를 들고 집으로 왔지만 이제는 마음이 가벼워 지는 것 같다고 고백했다. 사 교장이 지금은 형편이 어떠시냐고 묻자 노인은 세끼 밥은 해결하고 산다고 말했다고 한다. 이어 가난해서 배우지 못했다며 이 돈이 어려운 학생들에게 전달됐으면 좋겠다고 말하고 10여분 만에 학교를 떠났다. 사 교장은 노인이 마지막으로  \'이제는 마음이 후련하다\'고 말하더라며 전교생을 대상으로 추천을 받아 11월 중에 어려운 학생에게 장학금을 전달할 계획이라고 말했다.